In [142]:
import re
import urllib3
import certifi

In [163]:
#constract all regular expressions
title_regex=re.compile(r"class=\"title\"><h3>(.+)</h3>")
date_regex=re.compile(r"class=\"clamp-details\">\s+<span>(.+)</span>")
description_regex=re.compile(r"<div class=\"summary\">\s*([\S\s]+?)\s*</div>")
score_regex = re.compile(r"<span class=\"title\">Metascore:</span>\s+<a class=\"metascore_anchor\" href=\"/movie/.*?/critic-reviews\">\s+<div class=\"metascore_w large movie .+\">(.*?)</div>")
image_regex = re.compile(r"<a href=\"/movie/.*\"><img src=\"(.*)\" alt=\"")

# Construct an HTTP pool for connection
http=urllib3.PoolManager(ca_certs=certifi.where())


connect to mongoDB

In [160]:
import json
import pymongo
import certifi
#retrieve creditials to connect MongoDB
with open('/Users/natasha/Desktop/Fall 2022/DA320/Password MongoDB/credentials.json') as f:
    data = json.load(f)
    secret_key = data['mongodb']

# Connect to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile=certifi.where())
# Fetch the database named "DA-320"
da320_database = client['DA-320']

# Within the database we have "collections". Think of them as tables in SQL.
metacritic_data = da320_database['metacritic']

web scraping function

In [161]:
import pandas as pd
#retrive a list of movies from a particular year and page of Metacritic
def metacritic_scraper (year:int,page:int) -> pd.DataFrame:
    #fetch the webpage
   url = f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered?={year}&sort=desc&view=detailed&page={page}"
   # Construct python request pool
   response = http.request('GET',url,headers={'User-Agent':'Mozilla/5.0'})
   datastring = str(response.data, "utf-8")

   #execute all regular expression
   titles = title_regex.findall(datastring)
   dates = date_regex.findall(datastring)
   descriptions = description_regex.findall(datastring)
   scores = score_regex.findall(datastring)
   images = image_regex.findall(datastring)
   #return unified collection  
   print("title: " , len(titles), "date: " , len(dates), "description: " , len(descriptions), "score: " , len(scores), "image: " , len(images))
   dataset = {"title": titles, "date": dates, "description": descriptions, "score": scores, "image":images}
   return pd.DataFrame(dataset) 

Loop through years and pages

In [167]:
import re
import time

#Write a CVS file with this data
for year in range(2000,2023):
   page=0
   print(f"Collection date for {year} page {page}.....")

   #retry a page multiple times
   while True:
      data = metacritic_scraper(year,page)

      #stop when we reach a page with zero rows
      if len(data) == 0:
         break

      #conver the dataframe into a list of movies to insert into Mongo DB
      movies_to_insert =[]
      for row in data.itertuples():
         movie = {
            "title": row.title,
            "release_date": row.date,
            "description": row.description,
            "metascore": row.score,
            "image_url": row.image,
         }
         movies_to_insert.append(movie)
         
      #insert resords into MongoDB
      print(f"Inserting{len(movies_to_insert)} moview for year {year} page {page}")
      metacritic_data.insert_many(movies_to_insert)
      page = page+1


Collection date for 2000 page 0.....
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2000 page 0
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2000 page 1
title:  0 date:  0 description:  0 score:  0 image:  0
Collection date for 2001 page 0.....
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2001 page 0
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2001 page 1
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2001 page 2
title:  100 date:  100 description:  100 score:  100 image:  100
Inserting100 moview for year 2001 page 3
title:  75 date:  75 description:  75 score:  75 image:  75
Inserting75 moview for year 2001 page 4
title:  0 date:  0 description:  0 score:  0 image:  0
Collection date for 2002 page 0.....
title:  100 date:  100 description:  100 sc

Retrieving a collection from MongoDB

In [174]:
import json
import pymongo
import certifi
import pandas

# Retrieve credentials for SQL Server from our credentials file
with open('/Users/natasha/Desktop/Fall 2022/DA320/Password MongoDB/credentials.json') as f:
   data = json.load(f)
   secret_key = data['mongodb']

# Fetch the database named "DA-320"

client = pymongo.MongoClient (secret_key, tlsCAFile=certifi.where())
da320_database = client['DA-320']
metactritic_data = da320_database['metacritic_new_2']
metacritic = pandas.DataFrame(metacritic_data.find())